## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# change path to reflect your env
%cd / content/gdrive/MyDrive/final-ml/ML-Final-Project
! pip3 install -r requirements.txt

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/cs394n_project/CS394N/src')

In [2]:
import torch

import matplotlib.pyplot as plt
import numpy as np

from utils.nets import *
from utils.model_tools import *
from utils.ciivae_etc import *

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

seed = 420

random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Import data/Preprocess

## Train CI-iVAE

In [5]:
dim_x = 1 #TODO
dim_u = 1 #TODO
dim_z = 16
hid_dim = 1 #TODO

num_es = 100
batch_size = 256
num_workers = 32

beta = 0.01
weight_decay = 5e-6
init_lr = 5e-5
lr_milestones = [25, 50, 75]
lr_gamma = 0.5

dtype = torch.float32
M = 50
alpha_step = 0.025
fix_alpha = None

adam_betas = (0.5, 0.999)

prior, encoder, decoder = ConvCIiVAE(dim_x, dim_u, hid_dim, dim_z=16, prior_node_list=[128, 128],
            decoder_node_list=[4096, 4096],
            decoder_final_activation='sigmoid')

enc_optimizer = torch.optim.Adam(encoder.parameters(),
                                     betas=adam_betas,
                                     lr=init_lr,
                                     weight_decay=weight_decay)
gen_optimizer = torch.optim.Adam(list(decoder.parameters())
                                     + list(prior.parameters()),
                                     betas=adam_betas,
                                     lr=init_lr,
                                     weight_decay=weight_decay)

enc_scheduler = torch.optim.lr_scheduler.MultiStepLR(enc_optimizer,
                                                         milestones=lr_milestones,
                                                         gamma=lr_gamma)
gen_scheduler = torch.optim.lr_scheduler.MultiStepLR(gen_optimizer,
                                                         milestones=lr_milestones,
                                                         gamma=lr_gamma)

mse_criterion = torch.nn.MSELoss()

In [6]:
# define training log    
loss_names = ['loss', 'recon_loss_post', 'kl_post_prior',
                  'recon_loss_encoded', 'kl_encoded_prior', 'l2_penalty']
logs = {}
for datasetname in ['train', 'val']:
    logs[datasetname] = {}
    for loss_name in loss_names:
        logs[datasetname][loss_name] = []
summary_stats = []

In [10]:
# train loop

# TODO: training progress

for e in range(1, num_epochs + 1):
    print(f"Epoch {e}\n-------------------------------")
    
    # get dataloader setup
    for x_batch, u_batch in tqdm.tqdm(dataloader['train'],
                                         desc='[Epoch %d/%d] Training' % (epoch, num_epoch)):
        prior.train()
        encoder.train()
        decoder.train() 

        enc_optimizer.zero_grad()
        gen_optimizer.zero_grad()

        # forward step
        lam_mean, lam_log_var = prior(u_batch)
        z_mean, z_log_var = encoder(x_batch)
        post_mean, post_log_var = compute_posterior(z_mean, z_log_var, lam_mean, lam_log_var)
        post_sample = sampling(post_mean, post_log_var)
        encoded_sample = sampling(z_mean, z_log_var)

        epsilon = torch.randn((z_mean.shape[0], z_mean.shape[1], M))
        if device == 'cuda':
            post_sample = post_sample.cuda()
            encoded_sample = encoded_sample.cuda()
            epsilon = epsilon.cuda()

        fire_rate_post, obs_log_var = decoder(post_sample)
        fire_rate_encoded, _ = decoder(encoded_sample)

        # compute objective function
        obs_loglik_post = -torch.mean((fire_rate_post - x_batch)**2, dim=1)
        obs_loglik_encoded = -torch.mean((fire_rate_encoded - x_batch)**2, dim=1)

        kl_post_prior = kl_criterion(post_mean, post_log_var, lam_mean, lam_log_var)
        kl_encoded_prior = kl_criterion(z_mean, z_log_var, lam_mean, lam_log_var)

        elbo_post = obs_loglik_post - beta_kl_post_prior*kl_post_prior
        elbo_encoded = obs_loglik_encoded - beta_kl_encoded_prior*kl_encoded_prior

        z_mean_tiled = torch.tile(torch.unsqueeze(z_mean, 2), [1, 1, M])
        z_log_var_tiled = torch.tile(torch.unsqueeze(z_log_var, 2), [1, 1, M])
        z_sample_tiled = z_mean_tiled + torch.exp(0.5 * z_log_var_tiled) * epsilon

        post_mean_tiled = torch.tile(torch.unsqueeze(post_mean, 2), [1, 1, M])
        post_log_var_tiled = torch.tile(torch.unsqueeze(post_log_var, 2), [1, 1, M])
        post_sample_tiled = post_mean_tiled + torch.exp(0.5 * post_log_var_tiled) * epsilon

        log_z_density_with_post_sample = -torch.sum((post_sample_tiled - z_mean_tiled)**2/(2*torch.exp(z_log_var_tiled))+(z_log_var_tiled/2), dim=1)
        log_post_density_with_post_sample = -torch.sum((post_sample_tiled - post_mean_tiled)**2/(2*torch.exp(post_log_var_tiled))+(post_log_var_tiled/2), dim=1)
        log_z_density_with_z_sample = -torch.sum((z_sample_tiled - z_mean_tiled)**2/(2*torch.exp(z_log_var_tiled))+(z_log_var_tiled/2), dim=1)
        log_post_density_with_z_sample = -torch.sum((z_sample_tiled - post_mean_tiled)**2/(2*torch.exp(post_log_var_tiled))+(post_log_var_tiled/2), dim=1)

        if fix_alpha is not None:
            if fix_alpha == 0.0:
                loss = torch.mean(-elbo_post)
            elif fix_alpha == 1.0:
                loss = torch.mean(-elbo_encoded)
            else:
                ratio_z_over_post_with_post_sample = torch.exp(log_z_density_with_post_sample-log_post_density_with_post_sample)
                ratio_post_over_z_with_z_sample = torch.exp(log_post_density_with_z_sample-log_z_density_with_z_sample)
                skew_kl_post = torch.log(1.0/(fix_alpha*ratio_z_over_post_with_post_sample+(1.0-fix_alpha)))
                skew_kl_post = torch.abs(torch.mean(skew_kl_post, dim=-1))
                skew_kl_encoded = torch.log(1.0/(fix_alpha+(1.0-fix_alpha)*ratio_post_over_z_with_z_sample))
                skew_kl_encoded = torch.abs(torch.mean(skew_kl_encoded, dim=-1))
                loss = -fix_alpha*elbo_encoded-(1.0-fix_alpha)*elbo_post+fix_alpha*skew_kl_encoded+(1.0-fix_alpha)*skew_kl_post
        else:
            alpha_list = np.arange(alpha_step, 1.0, alpha_step)
            loss = torch.zeros((elbo_post.shape[0], len(alpha_list)))
            i = 0
            for alpha in alpha_list:
                ratio_z_over_post_with_post_sample = torch.exp(log_z_density_with_post_sample-log_post_density_with_post_sample)
                ratio_post_over_z_with_z_sample = torch.exp(log_post_density_with_z_sample-log_z_density_with_z_sample)
                skew_kl_post = torch.log(1.0/(alpha*ratio_z_over_post_with_post_sample+(1.0-alpha)))
                skew_kl_post = torch.abs(torch.mean(skew_kl_post, dim=-1))
                skew_kl_encoded = torch.log(1.0/(alpha+(1.0-alpha)*ratio_post_over_z_with_z_sample))
                skew_kl_encoded = torch.abs(torch.mean(skew_kl_encoded, dim=-1))
                loss[:, i] = -alpha*elbo_encoded-(1.0-alpha)*elbo_post+alpha*skew_kl_encoded+(1.0-alpha)*skew_kl_post
                i += 1
            loss, _ = torch.min(loss, dim = 1)

        loss = torch.mean(loss)
        loss.backward()

        enc_optimizer.step()
        gen_optimizer.step()


    prior.eval()
    encoder.eval()
    decoder.eval()
    for datasetname in ['train', 'val']:
        loss_cumsum, sample_size = 0.0, 0
        obs_loglik_post_cumsum, kl_post_prior_cumsum = 0.0, 0.0
        obs_loglik_encoded_cumsum, kl_encoded_prior_cumsum = 0.0, 0.0
        for x_batch, u_batch in tqdm.tqdm(dataloader[datasetname],
                                         desc='[Epoch %d/%d] Computing loss terms on %s' % (e, num_e, datasetname)):
            if device == 'cuda':
                x_batch, u_batch = x_batch.cuda(), u_batch.cuda()

            # forward step
            lam_mean, lam_log_var = prior(u_batch)
            z_mean, z_log_var = encoder(x_batch)
            post_mean, post_log_var = compute_posterior(z_mean, z_log_var, lam_mean, lam_log_var)
            post_sample = sampling(post_mean, post_log_var)
            encoded_sample = sampling(z_mean, z_log_var)

            epsilon = torch.randn((z_mean.shape[0], z_mean.shape[1], M))
            if device == 'cuda':
                post_sample = post_sample.cuda()
                encoded_sample = encoded_sample.cuda()
                epsilon = epsilon.cuda()

            fire_rate_post, obs_log_var = decoder(post_sample)
            fire_rate_encoded, _ = decoder(encoded_sample)

            # compute objective function
            obs_loglik_post = -torch.mean((fire_rate_post - x_batch)**2, dim=1)
            obs_loglik_encoded = -torch.mean((fire_rate_encoded - x_batch)**2, dim=1)

            kl_post_prior = kl_criterion(post_mean, post_log_var, lam_mean, lam_log_var)
            kl_encoded_prior = kl_criterion(z_mean, z_log_var, lam_mean, lam_log_var)

            elbo_pi_vae = obs_loglik_post - beta_kl_post_prior*kl_post_prior
            elbo_vae = obs_loglik_encoded - beta_kl_encoded_prior*kl_encoded_prior

            z_mean_tiled = torch.tile(torch.unsqueeze(z_mean, 2), [1, 1, M])
            z_log_var_tiled = torch.tile(torch.unsqueeze(z_log_var, 2), [1, 1, M])
            z_sample_tiled = z_mean_tiled + torch.exp(0.5 * z_log_var_tiled) * epsilon

            post_mean_tiled = torch.tile(torch.unsqueeze(post_mean, 2), [1, 1, M])
            post_log_var_tiled = torch.tile(torch.unsqueeze(post_log_var, 2), [1, 1, M])
            post_sample_tiled = post_mean_tiled + torch.exp(0.5 * post_log_var_tiled) * epsilon

            log_z_density_with_post_sample = -torch.sum((post_sample_tiled - z_mean_tiled)**2/(2*torch.exp(z_log_var_tiled))+(z_log_var_tiled/2), dim=1)
            log_post_density_with_post_sample = -torch.sum((post_sample_tiled - post_mean_tiled)**2/(2*torch.exp(post_log_var_tiled))+(post_log_var_tiled/2), dim=1)
            log_z_density_with_z_sample = -torch.sum((z_sample_tiled - z_mean_tiled)**2/(2*torch.exp(z_log_var_tiled))+(z_log_var_tiled/2), dim=1)
            log_post_density_with_z_sample = -torch.sum((z_sample_tiled - post_mean_tiled)**2/(2*torch.exp(post_log_var_tiled))+(post_log_var_tiled/2), dim=1)

            if fix_alpha is not None:
                if fix_alpha == 0.0:
                    loss = torch.mean(-elbo_post)
                elif fix_alpha == 1.0:
                    loss = torch.mean(-elbo_encoded)
                else:
                    ratio_z_over_post_with_post_sample = torch.exp(log_z_density_with_post_sample-log_post_density_with_post_sample)
                    ratio_post_over_z_with_z_sample = torch.exp(log_post_density_with_z_sample-log_z_density_with_z_sample)
                    skew_kl_post = torch.log(1.0/(fix_alpha*ratio_z_over_post_with_post_sample+(1.0-fix_alpha)))
                    skew_kl_post = torch.abs(torch.mean(skew_kl_post, dim=-1))
                    skew_kl_encoded = torch.log(1.0/(fix_alpha+(1.0-fix_alpha)*ratio_post_over_z_with_z_sample))
                    skew_kl_encoded = torch.abs(torch.mean(skew_kl_encoded, dim=-1))
                    loss = -fix_alpha*elbo_encoded-(1.0-fix_alpha)*elbo_post+fix_alpha*skew_kl_encoded+(1.0-fix_alpha)*skew_kl_post
            else:
                alpha_list = np.arange(alpha_step, 1.0, alpha_step)
                loss = torch.zeros((elbo_post.shape[0], len(alpha_list)))
                i = 0
                for alpha in alpha_list:
                    ratio_z_over_post_with_post_sample = torch.exp(log_z_density_with_post_sample-log_post_density_with_post_sample)
                    ratio_post_over_z_with_z_sample = torch.exp(log_post_density_with_z_sample-log_z_density_with_z_sample)
                    skew_kl_post = torch.log(1.0/(alpha*ratio_z_over_post_with_post_sample+(1.0-alpha)))
                    skew_kl_post = torch.abs(torch.mean(skew_kl_post, dim=-1))
                    skew_kl_encoded = torch.log(1.0/(alpha+(1.0-alpha)*ratio_post_over_z_with_z_sample))
                    skew_kl_encoded = torch.abs(torch.mean(skew_kl_encoded, dim=-1))
                    loss[:, i] = -alpha*elbo_encoded-(1.0-alpha)*elbo_post+alpha*skew_kl_encoded+(1.0-alpha)*skew_kl_post
                    i += 1
                loss, _ = torch.min(loss, dim = 1)
            loss = torch.mean(loss)
                
            loss_cumsum += loss.item()*np.shape(x_batch)[0]
            obs_loglik_post_cumsum += torch.mean(obs_loglik_post).item()*np.shape(x_batch)[0]
            kl_post_prior_cumsum += torch.mean(kl_post_prior).item()*np.shape(x_batch)[0]
            obs_loglik_encoded_cumsum += torch.mean(obs_loglik_encoded).item()*np.shape(x_batch)[0]
            kl_encoded_prior_cumsum += torch.mean(kl_encoded_prior).item()*np.shape(x_batch)[0]
            sample_size += np.shape(x_batch)[0]

        l2_penalty = 0.0
        for networks in [prior, encoder, decoder]:
            for name, m in networks.named_parameters():
                if 'weight' in name:
                    l2_penalty += 0.5*torch.sum(m**2)
        logs[datasetname]['loss'].append(loss_cumsum/sample_size)
        logs[datasetname]['recon_loss_post'].append(-obs_loglik_post_cumsum/sample_size)
        logs[datasetname]['kl_post_prior'].append(kl_post_prior_cumsum/sample_size)
        logs[datasetname]['recon_loss_encoded'].append(-obs_loglik_encoded_cumsum/sample_size)
        logs[datasetname]['kl_encoded_prior'].append(kl_encoded_prior_cumsum/sample_size)
        logs[datasetname]['l2_penalty'].append(l2_penalty.item())
        
    # save loss curves
    linestyles = ['solid', 'dashed']
    i = 0
    for dataset_name in ['train', 'val']:
        plt.plot(logs[dataset_name]['loss'][:], linestyle=linestyles[i],
                label=dataset_name)
        i += 1
    if e == 1:
        plt.legend()
    plt.xlabel('e')
    plt.ylabel('loss')
    plt.savefig('%s/loss_curves.pdf' % (result_path), dpi=600)
        
    # update models and logs if the best validation loss is updated
    current_val_loss = logs['val']['loss'][-1]
    best_val_loss = current_val_loss if e == 1 else np.minimum(best_val_loss, current_val_loss)
    if best_val_loss == current_val_loss:
        # update model and logs
        best_val_e = e
        os.makedirs('%s/' % result_path, exist_ok=True)
        torch.save({'prior': prior,
                    'encoder': encoder,
                    'decoder': decoder,
                    'logs': logs,
                    'num_e': num_e,
                    'batch_size': batch_size,
                    'num_worker': num_worker,
                    'seed': seed,
                    'beta': beta,
                    'Adam_beta1': Adam_beta1,
                    'Adam_beta2': Adam_beta2,
                    'weight_decay': weight_decay,
                    'init_lr': init_lr,
                    'lr_milestones': lr_milestones,
                    'lr_gamma': lr_gamma,
                    'dtype': dtype,
                    'M': M,
                    'alpha_step': alpha_step,
                    'fix_alpha': fix_alpha,
                    'result_path': result_path},
                    '%s/model.pth' % result_path)
    if e == num_e:
        # update logs
        saved_model = torch.load('%s/model.pth' % result_path)
        saved_model['logs'] = logs
        torch.save(saved_model, '%s/model.pth' % result_path)

    current_summary_stats_row = {}

    current_summary_stats_row['e'] = e
    current_summary_stats_row['best_val_e'] = best_val_e
    current_summary_stats_row['train_loss'] = logs['train']['loss'][-1]
    current_summary_stats_row['val_loss'] = logs['val']['loss'][-1]
    current_summary_stats_row['train_recon_loss_post'] = logs['train']['recon_loss_post'][-1]
    current_summary_stats_row['val_recon_loss_post'] = logs['val']['recon_loss_post'][-1]
    current_summary_stats_row['train_kl_post_prior'] = logs['train']['kl_post_prior'][-1]
    current_summary_stats_row['val_kl_post_prior'] = logs['val']['kl_post_prior'][-1]
    current_summary_stats_row['train_recon_loss_encoded'] = logs['train']['recon_loss_encoded'][-1]
    current_summary_stats_row['val_recon_loss_encoded'] = logs['val']['recon_loss_encoded'][-1]
    current_summary_stats_row['train_kl_encoded_prior'] = logs['train']['kl_encoded_prior'][-1]
    current_summary_stats_row['val_kl_encoded_prior'] = logs['val']['kl_encoded_prior'][-1]
    current_summary_stats_row['l2_penalty'] = logs['train']['l2_penalty'][-1]
        
    summary_stats.append(current_summary_stats_row)
    pd.DataFrame(summary_stats).to_csv('%s/summary_stats.csv' % result_path, index=False)
    
    
print("Done training Conv-CI-iVAE")


Epoch 1
-------------------------------


NameError: name 'dataloader' is not defined